In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from multiprocessing import Pool, cpu_count
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import glob, os

def im_multi(path):
    im_stats_im_ = Image.open(path)
    return [path, {'Stats': ImageStat.Stat(im_stats_im_), 'Size': im_stats_im_.size}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size_0'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Size'][0])
    im_stats_df['size_1'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Size'][1])
    print('size...')
    im_stats_df['sum_0'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].sum[0])
    im_stats_df['sum_1'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].sum[1])
    im_stats_df['sum_2'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].sum[2])
    print('sum...')
    im_stats_df['mean_0'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].mean[0])
    im_stats_df['mean_1'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].mean[1])
    im_stats_df['mean_2'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].mean[2])
    print('mean...')
    im_stats_df['rms_0'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].rms[0])
    im_stats_df['rms_1'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].rms[1])
    im_stats_df['rms_2'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].rms[2])
    print('var...')
    im_stats_df['var_0'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].var[0])
    im_stats_df['var_1'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].var[1])
    im_stats_df['var_2'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].var[2])
    print('stddev...')
    im_stats_df['stddev_0'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].stddev[0])
    im_stats_df['stddev_1'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].stddev[1])
    im_stats_df['stddev_2'] = im_stats_df['path'].map(lambda x: im_stats_d[x]['Stats'].stddev[2])
    print(len(im_stats_df))
    #add fun features here
    return im_stats_df

train = pd.read_csv('../input/Train/train.csv')
train['path'] = train.train_id.map(lambda x: '../input/Train/' + str(x) + '.jpg')
train['dpath'] = train.train_id.map(lambda x: '../input/TrainDotted/' + str(x) + '.jpg')
train = train[train.path.isin(glob.glob('../input/Train/*.jpg'))] #restrict
test = pd.read_csv('../input/sample_submission.csv')
test['path'] = test.test_id.map(lambda x: '../input/Test/' + str(x) + '.jpg')

print(len(train),len(test))

train = im_stats(train)
#test = im_stats(test)

colx = [c for c in train.columns if c not in ['train_id', 'path', 'dpath','adult_males', 'subadult_males', 'adult_females', 'juveniles', 'pups']]
coly = ['adult_males', 'subadult_males', 'adult_females', 'juveniles', 'pups']

dmean = train[colx].mean(axis=0)
dmedian = train[colx].mean(axis=0)
for c in colx:
    test[c] = int((dmean[c] + dmedian[c]) / 2)
    
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=6, n_jobs=-1, random_state=0, verbose=0)
rfr.fit(train[colx], train[coly])
print(mean_squared_error(train[coly], rfr.predict(train[colx]))**0.5)

pred = pd.DataFrame(rfr.predict(test[colx])).astype(int)
test[coly[0]] = pred[0]
test[coly[1]] = pred[1]
test[coly[2]] = pred[2]
test[coly[3]] = pred[3]
test[coly[4]] = pred[4]

test[['test_id','adult_males', 'subadult_males', 'adult_females', 'juveniles', 'pups']].to_csv('z01_submission01.csv', index=False)

img = glob.glob('../input/Train/*.jpg') + glob.glob('../input/TrainDotted/*.jpg')
plt.rcParams['figure.figsize'] = (12.0, 10.0)
for i in range(20):
        plt.subplot(5, 4, i+1)
        im = Image.open(img[i])
        plt.imshow(im); plt.axis('off')
        
        
        